<a href="https://colab.research.google.com/github/noble752/nsysuhomework/blob/main/HW_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW02 Web Crawler

## 題目

KKBOX 華語新歌日榜

## 摘要

本報告旨在透過網路爬蟲技術，收集KKBOX 華語新歌日榜的前十名，作為聆聽音樂的參考。

## 引言

KKBOX 作為台灣知名的音樂串流平台，提供每日更新的音樂排行榜，反映出當前大眾對新歌的關注程度。

本次爬蟲的目標是擷取 KKBOX 平台上「日榜華語新歌」的前十名資訊，包含歌曲名稱及演唱者，藉此掌握最新音樂動態，並為音樂推薦、趨勢分析等應用提供基礎資料。

## 方法

目標網站： KKBOX 日榜華語新歌排行榜頁面
https://kma.kkbox.com/charts/daily/newrelease?terr=tw&lang=tc

頁面結構：
該排行榜頁面列出了每日更新的華語新歌前幾名資訊，包含每首歌曲的歌名、演唱者等資料。該網站內容主要由 JavaScript 動態渲染生成，因此須透過瀏覽器模擬方式獲取完整資料。

工具與技術：

使用 Python 的 selenium 模組模擬瀏覽器載入網頁。

使用 BeautifulSoup 解析動態渲染後的 HTML 結構。

使用 Google Chrome 開發者工具輔助分析 DOM 結構與資料位置。

In [3]:
# 爬取資料程式碼
import requests
from bs4 import BeautifulSoup


url = 'https://kma.kkbox.com/charts/daily/newrelease?terr=tw&lang=tc'
headers ={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'lxml')
soup.title

<title>華語新歌日榜 - KKBOX 風雲榜</title>

In [7]:
pip install selenium


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 26.5 MB/s eta 0:00:00


In [8]:
pip install webdriver-manager


In [11]:
# 1. 安裝 Chrome 及 ChromeDriver
!apt update > /dev/null
!apt install -y chromium-chromedriver > /dev/null
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium > /dev/null

# 2. 設定 selenium options
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

options = Options()
options.add_argument('--headless')  # 背景執行
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.binary_location = '/usr/bin/chromium-browser'

# 3. 開啟網頁
driver = webdriver.Chrome(options=options)
driver.get('https://kma.kkbox.com/charts/daily/newrelease?terr=tw&lang=tc')

# 4. 等 JavaScript 載入
time.sleep(5)

# 5. 解析網頁
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

# 6. 抓取資料
songs = soup.find_all('li', class_='charts-list-row')[:10]

for idx, song in enumerate(songs):
    # 歌名
    title_tag = song.find('span', class_='charts-list-song')
    title = title_tag['title'] if title_tag else 'N/A'

    # 歌手
    artist_tag = song.find('span', class_='charts-list-artist')
    artist = artist_tag['title'] if artist_tag else 'N/A'

    print(f"{idx+1}. 🎵 歌名: {title}")
    print(f"    👤 歌手: {artist}")




W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file
1. 🎵 歌名: 最好的朋友
    👤 歌手: 韋禮安, 東海
2. 🎵 歌名: 兄弟們要進城
    👤 歌手: 頑童MJ116
3. 🎵 歌名: OGS
    👤 歌手: 頑童MJ116
4. 🎵 歌名: 有一种遗憾叫我们
    👤 歌手: 于冬然
5. 🎵 歌名: 數到十
    👤 歌手: 曾沛慈
6. 🎵 歌名: OH! - Chinese Version
    👤 歌手: GENBLUE 幻藍小熊
7. 🎵 歌名: 隔著螢幕想念你
    👤 歌手: 派偉俊
8. 🎵 歌名: 你給我的
    👤 歌手: 家家
9. 🎵 歌名: 一二三
    👤 歌手: 田馥甄
10. 🎵 歌名: 花季
    👤 歌手: 陳芳語


## 結果


- 數據存儲方式
  - 將數據存儲為 CSV 格式，方便後續分析。

In [12]:
# 展示資料程式碼
import csv
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd

# 設定 Selenium options
options = Options()
options.add_argument('--headless')  # 背景執行
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.binary_location = '/usr/bin/chromium-browser'

# 啟動瀏覽器
driver = webdriver.Chrome(options=options)
driver.get('https://kma.kkbox.com/charts/daily/newrelease?terr=tw&lang=tc')

# 等待網頁載入
time.sleep(5)

# 解析網頁
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

# 抓取前10名的歌曲資訊
songs = soup.find_all('li', class_='charts-list-row')[:10]

# 用來存儲歌名和歌手的列表
song_data = []

# 抓取每首歌的資料
for song in songs:
    # 歌名
    title_tag = song.find('span', class_='charts-list-song')
    title = title_tag['title'] if title_tag else 'N/A'

    # 歌手
    artist_tag = song.find('span', class_='charts-list-artist')
    artist = artist_tag['title'] if artist_tag else 'N/A'

    # 把歌名和歌手放進歌單列表
    song_data.append([title, artist])

# 使用 pandas 存儲為 CSV 格式
df = pd.DataFrame(song_data, columns=['歌名', '歌手'])
df.to_csv('kkbox_new_releases.csv', index=False, encoding='utf-8')

# 印出確認訊息
print("數據已經成功保存到 'kkbox_new_releases.csv'")


數據已經成功保存到 'kkbox_new_releases.csv'


## 問題與挑戰

- 透過 AJAX 取得資料：
目標網站的資料並非靜態呈現，而是透過 JavaScript 和 AJAX 動態載入。因此，單純的 requests 模組無法獲取完整資料。為了解決這個問題，我必須使用 selenium 來模擬瀏覽器行為，並等待網頁完成渲染，才能獲得所需的 HTML 內容。這需要額外的時間來設置和測試。

## 結論


- 本次作業成功抓取並分析了 KKBox 日榜華語新歌的前十名資訊，並提取了歌名和歌手等資料。透過使用 requests、BeautifulSoup 和 selenium 等技術，我克服了 AJAX 資料獲取的挑戰，成功獲得了動態載入的資料，並將其存儲為 CSV 格式，方便後續的分析與處理。
- 未來，該爬蟲可以進一步擴展至其他音樂平台或進行跨平台比較分析，以便更全面地了解不同平台的音樂趨勢。此外，也可以將分析結果與其他數據進行綜合分析，提供更深層次的音樂趨勢洞察。

## 參考文獻

* [KKBOX風雲榜](https://kma.kkbox.com/charts/daily/newrelease?terr=tw&lang=tc)